<a href="https://colab.research.google.com/github/poseidon2022/Program-Aided-Language-Model-Mimicry/blob/main/Program_Aided_Language_Model_Mimicry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the necessary libraries and dependencies
!pip install langchain_google_genai
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
# getting environmental variable from colab secret
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KE')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
NEO4J_URL = userdata.get('NEO4J_URL')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",
                             google_api_key = GOOGLE_API_KEY)

#Example generation
llm.invoke("Write me a poem about LangChain").content

"A chain of thought, a link of code,\nLangChain whispers, a heavy load\nLifted from minds, once bogged down low,\nWith data's depths, a constant flow.\n\nNo longer trapped in silos deep,\nInformation's secrets, it helps to reap.\nModels and data, hand in hand,\nA symphony conducted, on your command.\n\nFrom question answering, sharp and bright,\nTo chatbots engaging, day and night,\nSummarization's art, it can wield,\nAnd stories it spins, on imagination's field.\n\nBut power like this, a double-edged sword,\nUse it wisely, let ethics be your lord.\nFor bias can hide, in code's embrace,\nAnd truth be distorted, leaving no trace.\n\nSo wield LangChain well, with purpose in sight,\nUnleash its potential, bathed in reason's light.\nFor in human hands, this tool can inspire,\nTo reach for knowledge, and set the world on fire. \n"

In [ ]:
# using this for our in-memory contextual chatting and query storage parameter
messages = []

In [10]:
# a function to initialize our neo4j database and also run a query
from neo4j import GraphDatabase
def initialize_database(username, url, password):
  driver = GraphDatabase.driver(
      url, username, password
  )

  return driver

def run_query(query_code, driver):

  if isinstance(query_code, list):
      query_code = query_code[0]
  with driver.session() as session:
      results = session.run(query_code)
      result_list = [record for record in results]
      return result_list


In [14]:
# augmenting the user_query with examples to be used as a prompt for the model to generate
# a structured cypher query
def augment_prompt(user_query):
    augmented_prompt = f"""
    Input: Who acted in the movie Inception?
    Cypher: MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {{title: "Inception"}}) RETURN a.name

    Input: Which movies did Leonardo DiCaprio act in?
    Cypher: MATCH (a:Actor {{name: "Leonardo DiCaprio"}})-[:ACTED_IN]->(m:Movie) RETURN m.title

    Input: {user_query}
    Cypher:
    """
    return augmented_prompt


In [16]:
# demonstrating the results with an example query
query = "Who acted in the movie Inception?"
messages.append(("human", augment_prompt(query)))
response = llm.invoke(messages).content

response

'```cypher\nMATCH (a:Actor)-[:ACTED_IN]->(m:Movie {title: "Inception"}) RETURN a.name\n``` \n'